In [1]:
# Based partly on http://mariofilho.com/how-to-predict-multiple-time-series-with-scikit-learn-with-sales-forecasting-example/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# from rfpimp import *  - not easy to install on cluster

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

- This notebook takes as input two files for train and test data

### Define functions to fit + evaluate model

In [2]:
# Functions to use for prediction evaluation

def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))



In [3]:
# Train the model, measure accuracy, and error metrics

def make_model(X, y, X_valid, y_valid, n_estimators,
              max_features, min_samples_leaf, random_state):

    #global rf
    
    rf = RandomForestRegressor(n_estimators=n_estimators,
                           n_jobs=-1,
                           oob_score=True,
                           max_features=max_features, 
                           min_samples_leaf=min_samples_leaf,
                           random_state = random_state,
                           verbose = 1)

    #Inputting train dataset into the model
    rf.fit(X, y)

    return rf

In [4]:
# Using the model to make a prediction & Error Analysis for Random Forest 

def run_model(X_valid):
    #global n, h, oob_valid
    n = rfnnodes(rf)
    h = np.median(rfmaxdepths(rf))
    #global y_pred
    y_pred = rf.predict(X_valid)
    #global mae_valid, rmsle_valid, r2_score_valid
    mae_valid = mean_absolute_error(y_valid, y_pred)
    rmsle_valid = np.sqrt( mean_squared_error(y_valid, y_pred) )
    r2_score_valid = rf.score(X_valid, y_valid)
    
    oob_valid = rf.oob_score_

    print(f"RF OOB score {rf.oob_score_:.5f} using {n:,d} tree nodes {h} median tree height")
    print(f"Validation R^2 {r2_score_valid:.5f}, RMSLE {rmsle_valid:.5f}, MAE {mae_valid:.2f}")
    
    return y_pred,rmsle_valid,mae_valid,r2_score_valid, n, h, oob_valid

Dataset input block

- Features: area, building_type, temperature, + one-hot encoded hour, day of week, month of year

In [5]:
# Setting 'parse_dates' in this case parses both dates and times
# These files are too large to commit so they're uploaded locally under `/exploring_models` but not pushed
train_data = pd.read_csv('weather1_education_train.csv', parse_dates = ['timestamp'])
val_data = pd.read_csv('weather1_education_test.csv', parse_dates = ['timestamp'])

In [6]:
X, X_valid = train_data.drop(['electricity', 'building_name', 'primary_space_usage', 'hour', 'year', 'month','weekday', 'date', 'timestamp', 'Unnamed: 0'], axis=1), val_data.drop(['electricity', 'building_name', 'primary_space_usage', 'hour', 'year', 'month','weekday', 'date', 'timestamp', 'Unnamed: 0'], axis=1)
y, y_valid = train_data['electricity'], val_data['electricity']


In [7]:
X.head()

,area,TemperatureC,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,...,hour_23,wkday_0,wkday_1,wkday_2,wkday_3,wkday_4,wkday_5,wkday_6,PSU_PrimClass,PSU_UnivClass
0,2777.0,7.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,2777.0,5.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,2777.0,5.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,2777.0,6.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,2777.0,7.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [8]:
y.head()

0    5.0
1    5.2
2    5.2
3    5.5
4    6.0
Name: electricity, dtype: float64

In [93]:
# Looks ok
X.dtypes.head()

area            float64
TemperatureC    float64
month_1           int64
month_2           int64
month_3           int64
dtype: object

In [94]:
X.dtypes.tail()

wkday_4          int64
wkday_5          int64
wkday_6          int64
PSU_PrimClass    int64
PSU_UnivClass    int64
dtype: object

In [11]:
n_estimators=10 #Number of trees: tradeoff between accuracy and cost here
max_features='auto' #maximum number of features to train on
min_samples_leaf=5 #minimum number of data points to make a new leaf on a tree


In [14]:
rf = make_model(X, y, X_valid, y_valid, n_estimators,max_features, min_samples_leaf, random_state = 42)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   15.3s remaining:   10.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   23.6s finished
C:\Users\dmmma\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [15]:
run_model(X_valid)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


RF OOB score 0.88374 using 816,960 tree nodes 63.5 median tree height
Validation R^2 0.80350, RMSLE 13.74878, MAE 7.72


(array([20.19254152, 19.23924333, 19.66326789, ..., 10.52783618,
         9.84343293,  9.81426626]),
 13.74877873868953,
 7.719820946260225,
 0.8035022063988659,
 816960,
 63.5,
 0.8837430858707599)

In [16]:
# Characterize feature importance
feature_importances = pd.DataFrame(rf.feature_importances_,
                            index = X.columns,
                            columns=['importance']).sort_values('importance',ascending=False)

feature_importances.head(10)

,importance
area,0.599944
TemperatureC,0.040237
wkday_5,0.033156
wkday_6,0.026978
month_8,0.023914
hour_10,0.023389
hour_9,0.023298
hour_11,0.022908
hour_13,0.022723
hour_12,0.022358


### Tune hyperparameters

In [58]:
# Something appears wrong with the setup, beacuse the results are the same (!) at each step

def tune_maxf(X, y, X_valid, y_valid, ntrees, maxf_min, maxf_max, maxf_step = 0.1):

# Fix minleaf while tuning this
    minleaf = 1
    
    list_of_r2_valid = []
    list_of_rmsle_valid = []
    list_of_mae_valid = []
    
    list_of_maxf = np.arange(maxf_min, maxf_max, maxf_step ).tolist()

    for maxf in np.arange(maxf_min, maxf_max, maxf_step ):
       print(f"n_estimators={ntrees}, max_features={maxf}, min_samples_leaf={minleaf}")
       make_model(X, y, X_valid, y_valid, n_estimators = ntrees, max_features = maxf, min_samples_leaf = minleaf, 
                  random_state = 42)
    
       y_pred,rmsle_valid,mae_valid,r2_score_valid, n, h, oob_valid = run_model(X_valid)
        
       list_of_r2_valid.append(r2_score_valid)
       list_of_rmsle_valid.append(rmsle_valid)
       list_of_mae_valid.append(mae_valid)
    
    
    min1 = list_of_r2_valid.index(min(list_of_r2_valid))
    min2 = list_of_rmsle_valid.index(min(list_of_rmsle_valid))
    min3 = list_of_mae_valid.index(min(list_of_mae_valid ))
                                    
    tuned_maxf = list_of_maxf[min2]
    
    print(f"tuned_maxf ={tuned_maxf}")
    
    return tuned_maxf

In [60]:
tuned_maxf = tune_maxf(X, y, X_valid, y_valid, ntrees = 10, maxf_min = 0.1, maxf_max = 0.7, maxf_step = 0.2)

n_estimators=10, max_features=0.1, min_samples_leaf=1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    3.2s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.8s finished
C:\Users\dmmma\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


RF OOB score 0.88374 using 816,960 tree nodes 63.5 median tree height
Validation R^2 0.80350, RMSLE 13.74878, MAE 7.72
n_estimators=10, max_features=0.30000000000000004, min_samples_leaf=1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    5.1s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    7.6s finished
C:\Users\dmmma\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


RF OOB score 0.88374 using 816,960 tree nodes 63.5 median tree height
Validation R^2 0.80350, RMSLE 13.74878, MAE 7.72
n_estimators=10, max_features=0.5000000000000001, min_samples_leaf=1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    6.5s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    9.8s finished
C:\Users\dmmma\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


RF OOB score 0.88374 using 816,960 tree nodes 63.5 median tree height
Validation R^2 0.80350, RMSLE 13.74878, MAE 7.72
tuned_maxf =0.1


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


In [61]:

def tune_minleaf(X, y, X_valid, y_valid, ntrees, maxf, minleaf_min, minleaf_max, minleaf_step = 1):

    list_of_r2_valid = []
    list_of_rmsle_valid = []
    list_of_mae_valid = []
    
    list_of_minleaf = list(range(minleaf_min, minleaf_max, minleaf_step ))

    for minleaf in range(minleaf_min, minleaf_max, minleaf_step ):
       print(f"n_estimators={ntrees}, max_features={maxf}, min_samples_leaf={minleaf}")
       make_model(X, y, X_valid, y_valid, n_estimators = ntrees, max_features = maxf, min_samples_leaf = minleaf, 
                  random_state = 42)
    
       y_pred,rmsle_valid,mae_valid,r2_score_valid, n, h, oob_valid = run_model(X_valid)
        
       list_of_r2_valid.append(r2_score_valid)
       list_of_rmsle_valid.append(rmsle_valid)
       list_of_mae_valid.append(mae_valid)
    
    
    min1 = list_of_r2_valid.index(min(list_of_r2_valid))
    min2 = list_of_rmsle_valid.index(min(list_of_rmsle_valid))
    min3 = list_of_mae_valid.index(min(list_of_mae_valid ))
                                    
    tuned_minleaf = list_of_minleaf[min2]
    
    print(f"tuned_minleaf ={tuned_minleaf}")
    
    return tuned_minleaf

In [62]:
tuned_minleaf = tune_minleaf(X, y, X_valid, y_valid, ntrees = 5, maxf = 0.1, minleaf_min = 1, minleaf_max = 4, minleaf_step = 1)

n_estimators=5, max_features=0.1, min_samples_leaf=1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.1s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.3s finished
C:\Users\dmmma\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


RF OOB score 0.88374 using 816,960 tree nodes 63.5 median tree height
Validation R^2 0.80350, RMSLE 13.74878, MAE 7.72
n_estimators=5, max_features=0.1, min_samples_leaf=2


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.6s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.9s finished
C:\Users\dmmma\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


RF OOB score 0.88374 using 816,960 tree nodes 63.5 median tree height
Validation R^2 0.80350, RMSLE 13.74878, MAE 7.72
n_estimators=5, max_features=0.1, min_samples_leaf=3


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.4s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished
C:\Users\dmmma\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


RF OOB score 0.88374 using 816,960 tree nodes 63.5 median tree height
Validation R^2 0.80350, RMSLE 13.74878, MAE 7.72
tuned_minleaf =1


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


### Fit model with selected hyperparameters


In [90]:

def fit_tuned_rf(X, y, X_valid, y_valid, ntrees, random_state = 42):
    
    make_model(X, y, X_valid, y_valid, n_estimators = ntrees, 
                   max_features = tuned_maxf, min_samples_leaf = tuned_minleaf, 
                   random_state = 42)
    
    y_pred,rmsle_valid,mae_valid,r2_score_valid, n, h, oob_valid = run_model(X_valid)

    total_len = y_pred.shape[0]
    num_buildings = len(val_data['building_name'].unique())

    #print(total_len)
    #print(num_buildings)

    y_pred = y_pred.reshape((num_buildings, total_len//num_buildings))
    
    return y_pred

In [91]:
y_pred = fit_tuned_rf(X, y, X_valid, y_valid, ntrees = 10)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    3.1s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    4.6s finished
C:\Users\dmmma\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


RF OOB score 0.88374 using 816,960 tree nodes 63.5 median tree height
Validation R^2 0.80350, RMSLE 13.74878, MAE 7.72
166383
19


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


In [92]:
y_pred.shape

(19, 8757)